# Hate Speech Detection B

In [1]:
import requests
import pandas as pd

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=4, progress_bar=False)

import torch

from scipy.special import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/mezghani/miniconda3/envs/lsir/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model

### Load Tokenizer and Classifier

In [2]:
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-hate')

model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-hate')

### Load Model Label Mapping

In [3]:
mapping_link = f'https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/mapping.txt'

In [4]:
response = requests.get(mapping_link)

In [5]:
labels = {i: s.split('\t')[1] for i, s in enumerate(response.content.decode('utf-8').split('\n'))}

In [6]:
labels

{0: 'not-hate', 1: 'hate'}

## Load Data

### Load Tweets

In [ ]:
df_tweets = pd.read_parquet('data/tweets/en/english_tweets.parquet')
df_tweets

### Load News IDs

In [ ]:
news_ids = pd.read_csv('data/news/news_indexes.csv', header=None).values.reshape(-1)
news_ids

In [ ]:
len(news_ids)

### Filter News

In [ ]:
df_tweets['id'] = df_tweets['id'].astype(int)
df_tweets['is_news'] = df_tweets['id'].isin(news_ids)
df_tweets.head(10)

## Hate Speech Detection

### Pre-Processing

In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(' '):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return ' '.join(new_text)

In [ ]:
df_tweets['pre_process_text'] = df_tweets.text.str.replace(r'\s+', ' ', regex=True).parallel_apply(preprocess)

In [ ]:
texts = df_tweets.pre_process_text.values.tolist()
len(texts)

### Classification

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
model = model.to(device)

In [ ]:
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
step = 50
res = []

for i in tqdm(list(range(0, len(df_tweets), step))):
    j = min(i+step, len(df_tweets))
    print(i, j, len(df_tweets))
    
    _texts = texts[i:j]
    
    ### Tokenizing

    encoded_input = tokenizer(_texts, return_tensors='pt', padding=True).to(device)

    ### Hate Speech Classification

    output = model(**encoded_input)

    scores = softmax(output[0].detach().cpu().numpy(), axis=1)
    
    res.append(scores)
    
    del encoded_input
    del output
    torch.cuda.empty_cache()

In [ ]:
result_scores = pd.DataFrame(np.concatenate(res, axis=0), index=sample.index).rename(columns=id2label)['hate']

In [ ]:
df_tweets['hate_speech_b'] = result_scores

In [ ]:
df_tweets[['id', 'hate_speech_b']].to_parquet('data/hate_speech/hate_model_b.parquet')